# 오픈포즈 네트워크 구성 및 구현

In [1]:
import torch
import torch.nn as nn
from torch.nn import init
import torchvision

## OpenPoseNet 구현

In [ ]:
class OpenPoseNet(nn.Module):
    def __init__(self):
        super(OpenPoseNet, self).__init__()

        self.model0 = OpenPose_Feature()

        # PAFs 
        self.model1_1 = make_OpenPose_block("block1_1")
        self.model2_1 = make_OpenPose_block("block2_1")
        self.model3_1 = make_OpenPose_block("block3_1")
        self.model4_1 = make_OpenPose_block("block4_1")
        self.model5_1 = make_OpenPose_block("block5_1")
        self.model6_1 = make_OpenPose_block("block6_1")

        # confidence heatmap
        self.model1_2 = make_OpenPose_block("block1_2")
        self.model2_2 = make_OpenPose_block("block2_2")
        self.model3_2 = make_OpenPose_block("block3_2")
        self.model4_2 = make_OpenPose_block("block4_2")
        self.model5_2 = make_OpenPose_block("block5_2")
        self.model6_2 = make_OpenPose_block("block6_2")

    def forward(self, x):
        out1 = self.model0(x)

        out1_1 = self.model1_1(out1)
        out1_2 = self.model1_2(out1)

        out2 = torch.cat([out1_1, out1_2, out1], 1)
        out2_1 = self.model2_1(out2)
        out2_2 = self.model2_2(out2)

        out3 = torch.cat([out2_1, out2_2, out1], 1)
        out3_1 = self.model3_1(out3)
        out3_2 = self.model3_2(out3)

        out4 = torch.cat([out3_1, out3_2, out1], 1)
        out4_1 = self.model4_1(out4)
        out4_2 = self.model4_2(out4)

        out5 = torch.cat([out4_1, out4_2, out1], 1)
        out5_1 = self.model5_1(out5)
        out5_2 = self.model5_2(out5)

        out6 = torch.cat([out5_1, out5_2, out1], 1)
        out6_1 = self.model6_1(out6)
        out6_2 = self.model6_2(out6)


        # 손실 계산을 위한 스테이지 결과 저장
        save_for_loss = []
        save_for_loss.append(out1_1)
        save_for_loss.append(out1_2)
        save_for_loss.append(out2_1)
        save_for_loss.append(out2_2)
        save_for_loss.append(out3_1)
        save_for_loss.append(out3_2)
        save_for_loss.append(out4_1)
        save_for_loss.append(out4_2)
        save_for_loss.append(out5_1)
        save_for_loss.append(out5_2)
        save_for_loss.append(out6_1)
        save_for_loss.append(out6_2)

        return (out6_1, out6_2), save_for_loss